In [1]:
import jax
import jax.numpy as jnp
from jax import jit, vmap, grad
from jax import random
import optax
import numpy as np
import pickle

In [12]:
def init_network_params(sizes, key=random.PRNGKey(4)):
    def random_layer_params(m, n, key, scale=1e-2):
        w_key, b_key = random.split(key)
        return scale * random.uniform(w_key, (n, m), minval=1, maxval=3), scale * random.uniform(b_key, (n,), minval=1, maxval=3)

    keys = random.split(key, len(sizes))
    return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

def predict(NN, x):
    activations = x.reshape((-1, ))
    for w, b in NN[:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = jax.nn.sigmoid(outputs)
    final_w, final_b = NN[-1]
    logit = jnp.dot(final_w, activations) + final_b
    logit = 1/(1+jnp.exp(-logit))
    return logit.squeeze()

vpredict = vmap(predict, in_axes=(None, 0))
gpredict = grad(predict, argnums=1)
vgpredict = vmap(gpredict, in_axes=(None, 0))

In [13]:
def pi(x):
    return (1 / jnp.sqrt(2 * jnp.pi)) * jnp.exp(-0.5 * x**2)
vpi = vmap(pi, in_axes=(0,))

def pi_pre(NN, x):
    result = gpredict(NN, x)
    return result

@jit
def obj(NN, x_batch):
    def obj1(NN, x):
        return jnp.log(pi(x)) - jnp.log(pi_pre(NN, x))
    vobj1 = vmap(obj1, in_axes=(None, 0))

    result = vobj1(NN, x_batch)
    result = jnp.square(result)
    result = jnp.mean(result)
    return result

@jit
def evaluation(NN, test_zu):
    z_batch, u_batch = test_zu[:, 0], test_zu[:, 1]
    result = jnp.square(vpredict(NN, z_batch) - u_batch)
    result = jnp.mean(result)
    return result

In [ ]:
LAYER_SIZES = [1, 10, 10, 10, 1]
NN = init_network_params(LAYER_SIZES, random.PRNGKey(15))

optimizer= optax.adam(0.001)
opt_state = optimizer.init(NN)
test_zu = np.load('/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (TM)/test_zu.npy')
lowest = evaluation(NN, test_zu)
Lloss = []
Llowest = []
for epoch in range(2000000):
    x_batch = np.random.normal(0, 1, (10000, 1))

    grads = grad(obj)(NN, x_batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    NN = optax.apply_updates(NN, updates)

    Lloss.append(obj(NN, x_batch).item())
    Llowest.append(lowest.item())
    print(f"Epoch: {epoch}, loss:{obj(NN, x_batch)}, {lowest}")

    if  evaluation(NN, test_zu)<lowest:
        lowest = evaluation(NN, test_zu)
        NN_best = NN

    if epoch % 10000 == 0:
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (TM)/Lloss.npy', Lloss)
        np.save(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (TM)/Llowest.npy', Llowest)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (TM)/NN (best).pkl', 'wb') as f:
            pickle.dump(NN_best, f)
        with open(f'/content/drive/MyDrive/No. 22 Physica D (forward)/Normal checkpoints (TM)/NN_{epoch}.pkl', 'wb') as f:
            pickle.dump(NN, f)

流式输出内容被截断，只能显示最后 5000 行内容。
Epoch: 1858470, loss:3.2291352454194566e-06, 1.2861327280955237e-10
Epoch: 1858471, loss:3.7399768189061433e-06, 1.2861327280955237e-10
Epoch: 1858472, loss:3.82662938136491e-06, 1.2861327280955237e-10
Epoch: 1858473, loss:3.815453510469524e-06, 1.2861327280955237e-10
Epoch: 1858474, loss:4.14822306993301e-06, 1.2861327280955237e-10
Epoch: 1858475, loss:4.2205942918371875e-06, 1.2861327280955237e-10
Epoch: 1858476, loss:3.715473212650977e-06, 1.2861327280955237e-10
Epoch: 1858477, loss:3.342853460708284e-06, 1.2861327280955237e-10
Epoch: 1858478, loss:3.0402115953620523e-06, 1.2861327280955237e-10
Epoch: 1858479, loss:2.630825292726513e-06, 1.2861327280955237e-10
Epoch: 1858480, loss:2.1379876216087723e-06, 1.2861327280955237e-10
Epoch: 1858481, loss:9.78580715127464e-07, 1.2861327280955237e-10
Epoch: 1858482, loss:8.459579134978412e-07, 1.2861327280955237e-10
Epoch: 1858483, loss:6.063784780963033e-07, 1.2861327280955237e-10
Epoch: 1858484, loss:9.4647589321